In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os



In [3]:
src = "data/06_colima/colima.gpkg"
data = gpd.read_file(src)

data.head()

,CVEGEO,NOM_MUN,NOM_LOC,POB_TOTAL,$CO$,$HCHO$,$NO_2$,$O_3$,$SO_2$,GM_2020,...,P_15A17_F,P_15A17_M,P_18A24,P_18A24_F,P_18A24_M,P_15A49_F,P_60YMAS,P_60YMAS_F,AMBITO,geometry
0,0601000010288,Colima-VA,Ciudad de Villa de Álvarez,2557.0,1.000000,1.000000,1.000,1.000000,1.000,0.50,...,36,40,317,167,124,695,325,154,NaN,"POLYGON ((2318933.513 810638.283, 2319025.867 ..."
1,0600200010483,Colima-VA,Colima,742.0,0.977823,0.972755,0.885,0.965726,0.870,0.50,...,10,12,97,49,39,221,58,24,NaN,"POLYGON ((2319196.848 804654.646, 2319197.508 ..."
2,0600200010464,Colima-VA,Colima,90.0,0.639113,0.657921,0.577,0.641129,0.609,0.50,...,0,3,3,0,0,18,10,4,NaN,"POLYGON ((2320207.018 803190.824, 2320268.911 ..."
3,0601000010305,Colima-VA,Ciudad de Villa de Álvarez,1115.0,0.659274,0.648840,0.654,0.663306,0.630,0.00,...,18,10,156,62,65,360,80,31,NaN,"POLYGON ((2317155.357 810763.574, 2317160.203 ..."
4,0600200011320,Colima-VA,Colima,1520.0,0.597782,0.594349,0.558,0.584677,0.478,0.25,...,10,12,167,68,80,384,276,146,NaN,"POLYGON ((2315751.665 805258.386, 2315730.950 ..."


In [6]:
viviendas = pd.read_csv("data/06_colima/RESAGEBURB2020 - 06 Colima.csv")

# Turn ENTIDAD column into string with 2 digits (e.g. 01)
viviendas["ENTIDAD"] = viviendas["ENTIDAD"].apply(lambda x: str(x).zfill(2))

# Turn MUN column into string with 3 digits (e.g. 001)
viviendas["MUN"] = viviendas["MUN"].apply(lambda x: str(x).zfill(3))

# Turn LOC column into string with 4 digits (e.g. 0001)
viviendas["LOC"] = viviendas["LOC"].apply(lambda x: str(x).zfill(4))

# Create a new column called CVEGEO
viviendas["CVEGEO"] = viviendas["ENTIDAD"] + viviendas["MUN"] + viviendas["LOC"] + viviendas["AGEB"]


viviendas

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,AGEB,MZA,VIVTOT,POBTOT,CVEGEO
0,06,Colima,000,Total de la entidad Colima,0000,Total de la entidad,0000,0,288978,731391,0600000000000
1,06,Colima,001,Armería,0000,Total del municipio,0000,0,11236,27626,0600100000000
2,06,Colima,001,Armería,0001,Total de la localidad urbana,0000,0,5609,15368,0600100010000
3,06,Colima,001,Armería,0001,Total AGEB urbana,0081,0,560,1389,0600100010081
4,06,Colima,001,Armería,0001,Ciudad de Armería,0081,1,17,41,0600100010081
...,...,...,...,...,...,...,...,...,...,...,...
14737,06,Colima,010,Villa de Álvarez,0001,Ciudad de Villa de Álvarez,138A,4,0,0,060100001138A
14738,06,Colima,010,Villa de Álvarez,0001,Ciudad de Villa de Álvarez,138A,5,30,63,060100001138A
14739,06,Colima,010,Villa de Álvarez,0001,Ciudad de Villa de Álvarez,138A,6,10,14,060100001138A
14740,06,Colima,010,Villa de Álvarez,0001,Ciudad de Villa de Álvarez,138A,7,8,0,060100001138A


In [16]:
agebs = viviendas[viviendas["CVEGEO"].isin(data["CVEGEO"])]
agebs = agebs[agebs["NOM_LOC"] != "Total AGEB urbana"]

agebs = agebs[["CVEGEO", "MZA","VIVTOT","POBTOT"]]
agebs

,CVEGEO,MZA,VIVTOT,POBTOT
1069,0600200010464,1,18,44
1070,0600200010464,2,3,3
1071,0600200010464,4,12,29
1072,0600200010464,5,6,14
1090,0600200010483,1,37,109
...,...,...,...,...
12320,0601000010305,33,19,49
12321,0601000010305,34,0,0
12322,0601000010305,35,19,51
12323,0601000010305,36,11,15


In [17]:
agebs_grouped = agebs.groupby("CVEGEO").agg({"MZA":"count", "VIVTOT":"sum", "POBTOT":"sum"})
agebs_grouped

,MZA,VIVTOT,POBTOT
CVEGEO,,,
0600200010464,4,39,90
0600200010483,35,436,742
0600200011320,36,565,1520
0600500010108,49,771,1905
0600500010112,54,910,2186
0600500010235,57,643,1746
0600500180038,37,1064,3341
0600500180080,40,875,2639
0600700010460,40,891,2134


In [19]:
data

df = data[["CVEGEO", "NOM_MUN", "NOM_LOC"]]
df = df.merge(agebs_grouped, on="CVEGEO")
df

,CVEGEO,NOM_MUN,NOM_LOC,MZA,VIVTOT,POBTOT
0,0601000010288,Colima-VA,Ciudad de Villa de Álvarez,38,882,2557
1,0600200010483,Colima-VA,Colima,35,436,742
2,0600200010464,Colima-VA,Colima,4,39,90
3,0601000010305,Colima-VA,Ciudad de Villa de Álvarez,36,497,1115
4,0600200011320,Colima-VA,Colima,36,565,1520
5,0600500010235,Cuauhtémoc,Cuauhtémoc,57,643,1746
6,0600500180038,Cuauhtémoc,Quesería,37,1064,3341
7,0600500010112,Cuauhtémoc,Cuauhtémoc,54,910,2186
8,0600500180080,Cuauhtémoc,Quesería,40,875,2639
9,0600500010108,Cuauhtémoc,Cuauhtémoc,49,771,1905


In [20]:
df.to_excel("data/06_colima/agebs_colima_daniel.xlsx")